In [4]:
import pandas as pd
import os
from datetime import datetime
from memory_profiler import memory_usage
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
from symbol import Symbol
from index import Index
from portifolio import Portifolio
import seaborn as sns
from sklearn.preprocessing import scale
from scipy import stats


sns.set_style("dark")

%matplotlib inline

In [82]:
sym = Symbol()
ind = Index()
port = Portifolio()

ret = sym.get_symbol_data()
risk_free_rate = pd.read_hdf('constants.h5', 'risk_free_rate')
risk_free_rate_d = risk_free_rate['D'].resample('D').mean()
risk_free_rate_w = risk_free_rate['W'].resample('W').mean()
risk_free_rate_m = risk_free_rate['M'].resample('M').mean()
risk_free_rate_q = risk_free_rate['Q'].resample('Q').mean()
risk_free_rate_a = risk_free_rate['A'].resample('A').mean()


idata =sym.get_symbol_data('returns', 'infy', start='2016', end='2016-12-31').infy
index = ind.get_index_returns('nifty_50', start='2016', end='2016-12-31').nifty_50
RISK_FREE_RATE = np.log(1 + 0.075) / 250

In [18]:
def basic_regression(returns=idata, benchmark_returns=index,
                     risk_free_rate=RISK_FREE_RATE):
    symbol = returns.name
    benchmark = benchmark_returns.name
    returns = pd.DataFrame(returns)
    returns = returns.join(benchmark_returns)
    returns = returns.dropna(subset=[benchmark])
    returns = returns - risk_free_rate
    slope, intercept, r_value, _, _ = stats.linregress(
        y=returns[symbol], x=returns[benchmark]
    )
    std_dev = returns[symbol].std()
    regression_measures = pd.Series(
        [symbol, benchmark, intercept, slope, std_dev, r_value**2],
        index=['symbol', 'benchmark', 'alpha', 'beta', 'std_dev', 'r_square']
    )
    return regression_measures

def basic_regressions(returns=idata, benchmark_returns=index,
                      risk_free_rate=RISK_FREE_RATE):
    if not (isinstance(returns, pd.Series) and isinstance(benchmark_returns, pd.Series) ):
        raise KeyError
    returns = returns.dropna()
    benchmark_returns = benchmark_returns.dropna()
    common_index = np.intersect1d(returns.index, benchmark_returns.index)
    returns = returns[returns.index.isin(common_index)]
    benchmark_returns = benchmark_returns[benchmark_returns.index.isin(common_index)]
    returns = returns - risk_free_rate
    benchmark_returns = benchmark_returns - risk_free_rate
    slope, intercept, r_value, _, _ = stats.linregress(
        y=returns, x=benchmark_returns
    )
    std_dev = returns.std()
    regression_measures = pd.Series(
        [returns.name, benchmark_returns.name, intercept, slope, std_dev, r_value**2],
        index=['symbol', 'benchmark', 'alpha', 'beta', 'std_dev', 'r_square']
    )
    return regression_measures

In [83]:
def set_risk_free_rate(returns=idata, risk_free_rate=risk_free_rate):
    if isinstance(returns, pd.Series):
        returns = pd.DataFrame(returns)
    else:
        returns = returns.copy()
    freq = str(returns.index.inferred_freq)[0]
    if freq == 'N':
        if (returns.index[1] - returns.index[0]).days < 10:
            risk_free_rate = risk_free_rate_d
        else:
            risk_free_rate = risk_free_rate_a
    elif freq == 'W':
        risk_free_rate = risk_free_rate_w
    elif freq == 'M':
        risk_free_rate = risk_free_rate_m
    elif freq == 'Q':
        risk_free_rate = risk_free_rate_q
    elif freq == 'A':
        risk_free_rate = risk_free_rate_a
#     risk_free_rate = risk_free_rate[freq].resample(freq).first()
    for symbol in returns.columns:
        returns[symbol] = risk_free_rate
    return returns

In [90]:
# idata =sym.get_symbol_data('returns', 'infy').infy
# idata_a = idata.resample('A').sum()
%timeit set_risk_free_rate(idata_a)
set_risk_free_rate(idata_a)

1000 loops, best of 3: 472 µs per loop


,infy
date,
1996-12-31,0.058773
1997-12-31,0.058773
1998-12-31,0.058773
1999-12-31,0.058773
2000-12-31,0.058773
2001-12-31,0.058773
2002-12-31,0.058773
2003-12-31,0.048350
2004-12-31,0.046542


In [28]:
memf = memory_usage(basic_regressions)
memg = memory_usage(basic_regression)
maxmemf = max(memf)
meanf = np.mean(memf)
maxmemg = max(memg)
meang = np.mean(memg)


In [29]:
print(maxmemf, meanf, maxmemg, meang)

144.1484375 144.1484375 144.1484375 144.1484375
